# Route efficiency demo

This gives a demo of the analysis in the pre-print:
[Synthetic route design & assessment using vectors derived from similarity and complexity](https://chemrxiv.org/engage/chemrxiv/article-details/65f0268ce9ebbb4db98a984e)

It will use routes extracted from the US patent dataset created for the [PaRoutes](https://github.com/MolecularAI/PaRoutes) benchmark set.

In [1]:
#@title Installation -- Run this cell to install rnxutils

!pip install reaction-utils
!wget https://zenodo.org/record/7341155/files/ref_routes_n1.json?download=1 -O ref_routes_n1.json

  Cloning https://github.com/connorcoley/rdchiral.git (to revision master) to /scratch/kpzn768/pip-install-ni8guwwj/rdchiral_29c467ec6d094df59508ec64de80b753
  Running command git clone -q https://github.com/connorcoley/rdchiral.git /scratch/kpzn768/pip-install-ni8guwwj/rdchiral_29c467ec6d094df59508ec64de80b753
^C
ERROR: Operation cancelled by user
--2024-03-14 10:19:21--  https://zenodo.org/record/7341155/files/ref_routes_n1.json?download=1
Resolving proxy.srv.scp (proxy.srv.scp)... 10.96.66.4
Connecting to proxy.srv.scp (proxy.srv.scp)|10.96.66.4|:3128... connected.
Proxy request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/7341155/files/ref_routes_n1.json [following]
--2024-03-14 10:19:22--  https://zenodo.org/records/7341155/files/ref_routes_n1.json
Connecting to proxy.srv.scp (proxy.srv.scp)|10.96.66.4|:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 64187987 (61M) [text/plain]
Saving to: ‘ref_routes_n1.json’

 3% [>               

In [ ]:
import json
from collections import Counter
from operator import itemgetter

import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import Descriptors, rdFingerprintGenerator
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

from rxnutils.routes.base import SynthesisRoute

from complexity import calc_cm_star, calculate_molecular_complexity

np.seterr(divide='ignore')
sns.set_context("paper", font_scale=1.5)
sns.set_style("darkgrid")

### Read in PaRoutes n=1 routes (n = 10,000)

Read in the PaRoutes routes from the n=1 set and created route objects from them

In [ ]:
with open("ref_routes_n1.json", "r") as fileobj:
    route_dicts = json.load(fileobj)
routes = [SynthesisRoute(route_dict) for route_dict in route_dicts]
len(routes)

### Convert to chains and pandas Dataframe (n = 42,918)

This extracts individual chains from the synthesis routes and put each molecule of such a chain on a row in a pandas DataFrame

Uses CM* in the creation of the chains

In [ ]:
def make_df(route, project):
    chains = route.chains(calc_cm_star)
    df = pd.DataFrame([mol for chain in chains for mol in chain])
    df["project"] = project
    df["serial"] = df["project"] + "-" + df["serial"]
    df = df[
        [
            "project",
            "serial",
            "step",
            "chain",
            "reaction_id",
            "smiles",
            "type",
        ]
    ]
    return df.rename(
        columns={
            "serial": "idx",
            "step": "step_id",
            "smiles": "SMILES",
        }
    )
df = None
for idx, route in enumerate(tqdm(routes), 1):
    temp_df = make_df(route, f"USPTO-{idx:06}")
    if df is None:
        df = temp_df
    else:
        df = pd.concat([df, temp_df])
len(df)

Compare last route with the chain that were extracted

In [ ]:
routes[-1].image()

In [ ]:
df[df.project=="USPTO-010000"]

#### Select routes with < 3 chains (n = 42,857)

In [ ]:
subs = df[(df["chain"] != "sub1")&(df["chain"] != "lls")]
sub2_list = subs.project.tolist()
df = df[~df["project"].isin(sub2_list)]

subs = df[df["chain"] == "sub1"] 
sub1_list = subs.project.tolist()
df = df.assign(linear=np.where(df.project.isin(sub1_list), False, True))

len(df)

### Generate descriptors

Similarity to target
* Morgan r=2 bit vector
* Tanimoto

In [ ]:
# Generate RDKit mol objects
df["mol"] = [Chem.MolFromSmiles(smi) for smi in df["SMILES"]]

# Generate Morgan fingerprints with and without counts
mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=2048)
df["morg"] = [mfpgen.GetFingerprint(mol) for mol in df["mol"]]

# Calculate Tanimoto similarity metrics for all chains
def calc_similarity(df, fp, metric):
    target_fp = df[df["type"]=="target"][fp].iloc[0]
    return metric(target_fp, df[fp].to_list())

tan = df.groupby("project").apply(
    calc_similarity, fp="morg", metric=DataStructs.BulkTanimotoSimilarity
).explode()
df = df.assign(tan=tan.values)

# Drop fingerprint columns
df = df.drop(columns=["morg"])

CSE, CM, and CM*

In [ ]:
def calc_complexity(mol):
    dict_ = dict(zip(["CSE", "CM", "CM*"], calculate_molecular_complexity(mol)))
    return pd.Series(dict_)
metrics = df["mol"].apply(calc_complexity)
df = df.assign(**metrics.to_dict(orient="series"))

In [ ]:
df["CM*"] = df.SMILES.apply(calc_cm_star)
print(df["CM*"].describe())
      
df["nC"] = (df["CM*"] - df["CM*"].min()) / (df["CM*"].max() - df["CM*"].min())
print(df.nC.describe())

Molecular weight

In [ ]:
df["MW"] = [Chem.Descriptors.MolWt(mol) for mol in df["mol"]]
df["MW"] = df.MW.round(0).astype(int)

#### Remove low-MW targets and low complexity mols (eg, isotope labelled targets and erroneous route compilations, n = 42, 854)

In [ ]:
# low-MW targets
low_mw = (df["type"] == "target") & (df["MW"] < 120)
df_low_mw = df[low_mw]
low_mw_projs = df_low_mw.project.tolist()

# low complexity (CM*) SMs or inters

low_comp = (df["CM*"] < 2.6)
df_low_comp = df[low_comp]
low_comp_projs = df_low_comp.project.tolist()

remove_list = low_mw_projs + low_comp_projs
keep_sel = ~(df["project"].isin(remove_list))
df = df[keep_sel]

df.reset_index(inplace=True)
len(df)

### Calculate dx/dy values for required x/y variables

This calculates the delta values along the chains for the fingerprints, and the complexity metrics

In [ ]:
columns = ["tan", "MW", "CM*", "nC"]
df = df.sort_values(by=["project", "chain", "step_id"], ignore_index=True)
for col in columns:
    # Calculate dx and dy values along LLS for each project
    df[f"d_{col}"] = df.groupby("project")[col].diff().fillna(0)
df.head(5)


### Calculate necessary values to assess vector path real and vector path perfect

Will use Tanimoty similarity of bit-vector and nC as the two axis, but this can be changed

In [ ]:
x_var = "tan"
y_var = "nC"

df["proj_chain"] = np.where(df["chain"] == "lls", df["project"], df["project"] + "x")

# Generate new dataframes of x_var aggregates and y_var aggregates grouped by project
# First and last will return the SM and target for each project
data_aggs_x = df.groupby("proj_chain", group_keys=False)[x_var].agg(["first", "last"])
data_aggs_y = df.groupby("proj_chain", group_keys=False)[y_var].agg(["first", "last"])

#  Map summed values from aggregate dataframes to original dataframe
# First/last group values used to calculate ranges from SM to target
df["x_range"] = df["proj_chain"].map(data_aggs_x["last"]) - df["proj_chain"].map(
    data_aggs_x["first"]
)
df["y_range"] = df["proj_chain"].map(data_aggs_y["last"]) - df["proj_chain"].map(
    data_aggs_y["first"]
)

df["vmin"] = np.sqrt((df["x_range"] ** 2 + df["y_range"] ** 2).astype(float))

The scalar projection between the xy vectors and vmin

In [ ]:
df["vector"] = list(zip(df["d_tan"], df["d_nC"]))
df["vector_vmin"] = list(zip(df.x_range, df.y_range))    
df["sc_pr_vmin"] = [(np.dot(v1, v2) / np.linalg.norm(v2)) for v1, v2 in list(zip(df.vector, df.vector_vmin)) ]

The maximum step count per project

In [ ]:
targets = df.groupby("project")["step_id"].agg([max])
df["step_max"] = df["project"].map(targets["max"])
df["step_max_cat"] = df["step_max"]
df = df.astype({"step_max_cat":"category"})

### Fit a polynomial function to the step count versus median vmin

This will be used as a function to calculate route efficiency

In [ ]:
steps = []
meds = []
for i in range(2, df["step_max"].max()+1):
    filters = np.where((df["step_max"] == i) & (df["type"] == "target"))
    dfilt_vmin_iqs = df.loc[filters]

    steps.append(i)
    meds.append(dfilt_vmin_iqs.vmin.median())
    
inv_steps = [1/i for i in steps]
steps_array = np.array(inv_steps)
meds_array = np.array(meds)

poly = PolynomialFeatures(degree=2, include_bias=False)
poly_features = poly.fit_transform(steps_array.reshape(-1, 1))

poly_reg_model = LinearRegression()
poly_reg_model.fit(poly_features, meds_array)

a = poly_reg_model.coef_[0]
b = poly_reg_model.coef_[1]
c = poly_reg_model.intercept_

df["eff_R"] = df["vmin"] / ((a / df["step_id"]) + ( b / df["step_id"]**2) + c)
print(f'Regression formula = a/n + b/n**2 + c')
print(f' a = {a}')
print(f' b = {b}')
print(f' c = {c}')

Regression coefficient from fitting of litterature values

In [ ]:
a2 = -0.40330893356344394
b2 = -0.009805775251438161
c2 = 0.9435001177152702

In [ ]:
sns.scatterplot(x=steps, y=meds_array)
sns.lineplot(x=steps, y=a/np.asarray(steps) + b/np.asarray(steps)**2 + c, label="patent")
sns.lineplot(x=steps, y=a2/np.asarray(steps) + b2/np.asarray(steps)**2 + c2, label="litt.").set(xlabel="step (LLS)", ylabel="$v_{min}$")

### Calculate the categories for the transformation efficiency

In [ ]:
filters = np.where((df.type != "sm") & (df["type"] != "branch"))
dfilt = df.loc[filters]
quantiles = dfilt.sc_pr_vmin.quantile([0.2, 0.4, 0.6, 0.8]).tolist()
print(dfilt.sc_pr_vmin.describe(percentiles=[0.2, 0.4, 0.6, 0.8]))

df["prod_cat"] = "v-low"
df.loc[df.sc_pr_vmin>quantiles[0],"prod_cat"] = "low"
df.loc[df.sc_pr_vmin>quantiles[1],"prod_cat"] = "medium"
df.loc[df.sc_pr_vmin>quantiles[2],"prod_cat"] = "high"
df.loc[df.sc_pr_vmin>quantiles[3],"prod_cat"] = "v-high"
df.prod_cat = df.prod_cat.astype({"prod_cat": "category"})

### Plotting example routes

The route index can be set with the `idx` variable

In [ ]:
idx = 5000
sel_data = df[df["project"] == f"USPTO-{idx:06}"]

x_var="tan"
y_var="nC"


df_tgt = sel_data[sel_data.type=="target"]
vmin = df_tgt["vmin"].values[0]
eff_r = df_tgt["eff_R"].values[0]
print("Synthetic range = ", vmin)
print("Route efficiency = ", eff_r)

sims = list(sel_data[x_var])
comps = list(sel_data[y_var])
prods = list(sel_data["prod_cat"])
types = list(sel_data["type"])

sns.lineplot(
    data=sel_data, x=x_var, y=y_var, zorder=1, sort=False, dashes=True
)
sns.scatterplot(
    data=sel_data.iloc[:1],
    x=x_var,
    y=y_var,
    markers={"sm":"s"},
    style="type",
    hue="type",
    s=60,
)

sns.scatterplot(
    data=sel_data.iloc[1:],
    x=x_var,
    y=y_var,
    #markers={"sm":"s"},
    size="prod_cat",
    size_order = ["v-high", "high", "medium", "low", "v-low"],
    sizes = (35, 85),
    hue="prod_cat",
    hue_order = ["v-high", "high", "medium", "low", "v-low"],
    style = "prod_cat",
    style_order = ["v-high", "high", "medium", "low", "v-low"],
    markers = {"v-high": "^", "high": "^", "medium": "o", "low":"v", "v-low": "v"},   
    s=60,
).set_xlabel("similarity to target (S)", fontsize=12, fontweight="bold")

In [ ]:
routes[idx-1].image()

### Scoring routes

This shows how one could implement a scoring function for a `SynthesisRoute` object

First we setup 3 different functions:
1. A molecular complexity scorer
2. A function that calculates similarities
3. A function that calculate V_min50

In [ ]:
# This function calculate the normalized CM*
def norm_cm_star(smiles, min_val=3.58, max_val=9.20):
    cstar = calc_cm_star(smiles)
    return (cstar - min_val) / (max_val - min_val)

# This function calculates the Tanimoty similarity between the targets and all the other molecules in the chain
def calc_similarities(smiles_list):
    rd_mols = [Chem.MolFromSmiles(smi) for smi in smiles_list]
    mfpgen = Chem.rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=2048)
    fps = [mfpgen.GetFingerprint(mol) for mol in rd_mols]
    return DataStructs.BulkTanimotoSimilarity(fps[-1], fps)

# This function calculate the V_min50
def vmin_step_func(nsteps, a = -1.3073, b=1.3267, c=1.0162):
    return a/nsteps + b/nsteps**2 + c

This now calculates the efficiency using the three functions above

In [ ]:

route = routes[idx-1]
chains = route.chains(norm_cm_star)
lls = chains[0]

smiles_list = [mol["smiles"] for mol in lls]
similarities = calc_similarities(smiles_list)
complexities = [mol["complexity"] for mol in lls]

synthetic_range = np.sqrt(
    (similarities[0] - similarities[-1]) ** 2
    + (complexities[0] - complexities[-1]) ** 2
)
efficiency = synthetic_range / vmin_step_func(len(lls)-1)
print("Synthetic range = ", synthetic_range)
print("Route efficiency = ", efficiency)